# **Setting up PySpark**

In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum as _sum, max as _max, min as _min
from pyspark.sql.types import *

# **Initialize Spark Session**

In [ ]:
# Create Spark session
spark = SparkSession.builder \
    .appName("Online Banking Analysis") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# **Load and Analyze Loan Data (loan.csv)**

In [ ]:
# Load loan data
loan_df = spark.read.csv("loan.csv", header=True, inferSchema=True)

# Show schema and sample data
loan_df.printSchema()
loan_df.show(5)

In [ ]:
# Load the data
df = spark.read.option("header", "true") \
               .option("inferSchema", "true") \
               .csv("loan.csv")

df.printSchema()

# Show first few rows
df.show(5)

# Clean column names
for col_name in df.columns:
    new_col_name = col_name.replace(" ", "_").replace("-", "_").replace(",", "")
    df = df.withColumnRenamed(col_name, new_col_name)

# 1. Number of loans in each category
loan_category_count = df.groupBy("Loan_Category").count().orderBy("count", ascending=False)
print("Number of loans in each category:")
loan_category_count.show()

In [ ]:
# 2. Number of people who have taken more than 1 lakh loan
df = df.withColumn("Loan_Amount_Clean",
                  regexp_replace(col("Loan_Amount"), ",", "").cast("double"))
loan_over_1lakh = df.filter(col("Loan_Amount_Clean") > 100000).count()
print(f"Number of people who have taken more than 1 lakh loan: {loan_over_1lakh}")

# 3. Number of people with income greater than 60000 rupees
high_income_count = df.filter(col("Income") > 60000).count()
print(f"Number of people with income greater than 60000 rupees: {high_income_count}")

In [ ]:
# 4. Number of people with 2 or more returned cheques and income less than 50000
returned_cheques_low_income = df.filter((col("_Returned_Cheque") >= 2) & (col("Income") < 50000)).count()
print(f"Number of people with 2+ returned cheques and income < 50000: {returned_cheques_low_income}")

# 5. Number of people with 2 or more returned cheques and are single
returned_cheques_single = df.filter((col("_Returned_Cheque") >= 2) & (col("Marital_Status") == "SINGLE")).count()
print(f"Number of people with 2+ returned cheques and are single: {returned_cheques_single}")

In [ ]:
# 6. Number of people with expenditure over 50000 a month
high_expenditure_count = df.filter(col("Expenditure") > 50000).count()
print(f"Number of people with expenditure over 50000 a month: {high_expenditure_count}")

# 7. Number of members who are eligible for credit card
credit_card_eligible = df.filter((col("Income") > 50000) &
                                (col("Expenditure") < col("Income")/2) &
                                (col("Overdue") == 0)).count()
print(f"Number of members eligible for credit card: {credit_card_eligible}")

# **Load and Analyze Credit card Data (credit card.csv)**

In [ ]:
# Create Spark session
spark = SparkSession.builder \
    .appName("CreditCardAnalysis") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

# Load the CSV file (upload it first to Colab)
df = spark.read.csv("credit card.csv", header=True, inferSchema=True)

# Show the schema to understand the data structure
df.printSchema()

In [ ]:
# Credit card users in Spain
spanish_users = df.filter(col("Geography") == "Spain")
spanish_users_count = spanish_users.count()
print(f"Number of credit card users in Spain: {spanish_users_count}")

spanish_users.select("CustomerId", "Surname", "CreditScore", "Gender", "Age").show(10)

In [ ]:
#  Number of members who are eligible and active in the bank
active_eligible_members = df.filter(
    (col("CreditScore") > 650) &
    (col("IsActiveMember") == 1))

active_eligible_count = active_eligible_members.count()
print(f"Number of active and eligible members: {active_eligible_count}")

# Show some active eligible members
active_eligible_members.select("CustomerId", "Surname", "CreditScore", "IsActiveMember", "Geography").show(10)

# **Load and Analyze Transaction Data (txn.csv)**

In [ ]:

# Create Spark session
spark = SparkSession.builder \
    .appName("BankTransactionAnalysis") \
    .getOrCreate()

# Load the CSV file
df = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .csv("txn.csv")

# Show the schema to understand the data structure
df.printSchema()

# Let's see some sample data
df.show(5, truncate=False)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum as _sum, max as _max, min as _min
from pyspark.sql.types import *

# 1. Maximum withdrawal amount in transactions
_max_withdrawal = df.agg(max(col(" WITHDRAWAL AMT "))).collect()[0][0]
print(f"Maximum withdrawal amount: {_max_withdrawal}")

# 2. Minimum withdrawal amount of an account
_min_withdrawal = df.agg(min(col(" WITHDRAWAL AMT "))).collect()[0][0]
print(f"Minimum withdrawal amount: {_min_withdrawal}")

# 3. Maximum deposit amount of an account
_max_deposit = df.agg(max(col(" DEPOSIT AMT "))).collect()[0][0]
print(f"Maximum deposit amount: {_max_deposit}")

# 4. Minimum deposit amount of an account
_min_deposit = df.agg(min(col(" DEPOSIT AMT "))).collect()[0][0]
print(f"Minimum deposit amount: {_min_deposit}")

In [ ]:
# 5. Sum of balance for every bank account
balance_sum = df.agg(sum(col("BALANCE AMT"))).collect()[0][0]
print(f"Total sum of all balances: {balance_sum}")

# 6. Number of transactions on each date
transactions_per_date = df.groupBy("VALUE DATE").count().orderBy("VALUE DATE")
print("Number of transactions per date:")
transactions_per_date.show(truncate=False)

# 7. List of customers with withdrawal amount more than 1 lakh
high_withdrawal_customers = df.filter(col(" WITHDRAWAL AMT ") > 100000) \
    .select("Account No", " WITHDRAWAL AMT ", "VALUE DATE") \
    .distinct()
print("Customers with withdrawal amount more than 1 lakh:")
high_withdrawal_customers.show(truncate=False)

# Submittted by
**Siva balan T**